In [37]:
import math
import requests


In [38]:
paid_at_from = '2025-01-01T00:00:00-03:00'
paid_at_to = '2025-01-31T23:59:59-03:00'

In [39]:
headers = {"accept": "application/json"}

In [40]:
logfile = 'iugu.csv'

In [41]:
cabecalho = "Código;Situação;Código da Assinatura;Data do pagamento;CPF/CNPJ;Nome;Total"

In [42]:
#paid, refunded, partially_refunded, chargeback
status = 'refunded'

In [ ]:
url = "https://api.iugu.com/v1/invoices?limit=1&start=0&paid_at_from=" + str(paid_at_from) + "&paid_at_to=" + str(paid_at_to) + "&status_filter=" +str(status) + "&api_token=xxxx"

response = requests.get(url, headers=headers)
print (response.text)

In [44]:
data = response.json()
totalItems = data['totalItems']
totalItems

18

In [45]:
paginas = math.ceil(totalItems/100) + 1
paginas

2

In [46]:
limit = 0

In [47]:
with open(logfile, 'w') as file:
  file.write(cabecalho+'\n')
  for i in range(0,paginas):
    url = "https://api.iugu.com/v1/invoices?limit=100"+ "&start=" + str(limit) + "&paid_at_from=" + str(paid_at_from) + "&paid_at_to=" + str(paid_at_to) + "&status_filter=" +str(status) +"&api_token=DDD91DBD410EBCE75AC818E48BB410B1EDF2401F344D8135B30D2DAF05D5D5C2"
    result = requests.get(url, headers=headers)
    dt = result.json()
    dados = dt["items"]
    for valor in dados:
      file.write(valor['id']+';'+ valor['status']+';' + str(valor['subscription_id'])+';' + valor['paid_at']+';' + str(valor['total_paid_cents'])+ '\n')
    limit += 100


In [48]:
file = open("iugu.csv","r")
Counter = 0
Content = file.read()
CoList = Content.split("\n")

for i in CoList:
    if i:
        Counter += 1

print(f'This is the number of lines in the file:{Counter}')

This is the number of lines in the file:19


##processa arquivo

In [ ]:
import requests
import json
import csv
import time

In [ ]:
url = 'https://afya-pebmed-refact.gateway.linkapi.solutions/v3/iugu'


In [ ]:
headers = {
  'Content-Type': 'application/json'
}

In [ ]:
logfile = 'processamento.txt'

In [ ]:
payload = json.dumps({
  "event": "invoice.status_changed",
  "data": {
    "id": "idfat",
    "status": "sts",
    "account_id": "0B864B780D28411F91D0172DAB585A2B",
    "async_charged": "",
    "subscription_id": "idsubs",
    "payment_method": "iugu_credit_card",
    "paid_at": "dtpaid",
    "paid_cents": "vlr"
  },
  "retry_count": "0"
})

In [ ]:
search_value = 'ReplyError: ERR invalid expire time in setex'

In [ ]:
i = 0

In [ ]:
with open(logfile, 'w') as file:
  with open('iugu.csv', 'r') as txt_file:
      for line_number, content in enumerate(txt_file):
          if line_number:  # pula cabeçalho
              colunas = content.strip().split(';')
              request = url
              body = payload.replace("idfat",colunas[0])
              body1 = body.replace("idsubs",colunas[2])
              body2 = body1.replace("dtpaid",colunas[3])
              body3 = body2.replace("vlr",colunas[6])
              body4 = body3.replace("sts",colunas[1])
              print(body4)

              response = requests.request("POST", url, headers=headers, data=body4)
              print(f'Resposta - OK: {response.text}')
              if search_value in response.text:
                response = requests.request("POST", url, headers=headers, data=body4)
                print(f'Resposta - Erro - segunda tentativa: {response.text}')
                if search_value in response.text:
                  response = requests.request("POST", url, headers=headers, data=body4)
                  print(f'Resposta - Erro - terceira tentativa: {response.text}')
              file.write(f'{colunas[0]};{body4};{response.text}\n')

              time.sleep(1)
              i += 1
              print(i)
